In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_desc(soup):

    try:
        # Outer Tag Object  css-cmh3n9
        title = soup.find("span", attrs={"class":'css-cmh3n9'})
        
        # Inner NavigatableString Object
        title_value = title.text
        # Title as a string value
        title_string = title_value.strip()
        print(title_string)
        
    except AttributeError:
        title_string = ""

    return title_string

In [19]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({
        'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
        'Accept-Language' : 'en-US , en;q=0.5 ',
    })
    
    
    URL = 'https://www.nykaafashion.com/catalogsearch/result/?q=tops&searchType=ManualSearch&internalSearchTerm=tops&typedSearchTerm=tops&p=2'
    d = {"Discription":[], "url":[],"Vendor":[]}
    
    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)
    
    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    # Fetch links as List of Tag Objects  css-19nu305
    links = soup.find_all('a',attrs={'class':'css-19nu305'})
    
    # Store the links
    links_list = []
    
    # Loop for extracting links from Tag Objects
    for link in links:
        links_list.append(link.get('href'))
    
    # Loop for extracting product details from each link 
    for link in links_list:
        prod_link = "https://www.nykaafashion.com" + link
        new_webpage = requests.get(prod_link , headers=HEADERS)
        
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        
        # Function calls to display all necessary product information
        d['Discription'].append(get_desc(new_soup))
        d['url'].append(prod_link)
        print(prod_link)
        d['Vendor'].append('NykaaFashion')
        
    nykaa_df = pd.DataFrame.from_dict(d)
    nykaa_df['Discription'].replace('', np.nan, inplace=True)
    nykaa_df = nykaa_df.dropna(subset=['Discription'])
    

In [20]:
nykaa_df

,Discription,url,Vendor


In [18]:
nykaa_df.to_csv("nykaa_tops.csv", header=True, index=False)